# Simple loop agent

In this example, we will create an loop agent that simply generate a description base on a same query and get all results

In other word, this is a "for loop" agent.

In [1]:
import sys
sys.path.append("../../src/")

In [2]:
from agent_design_pattern.agent import AgentMessage, BaseAgent
from agent_design_pattern.orchestration import LoopAgent
from chain import CasualOllamaSingleTurnChain

In [3]:
class Agent(BaseAgent):
    def __init__(self, chain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.chain = chain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.name
        return message

In [4]:
from typing import Generator


system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")

def iterator(max_iteration: int) -> Generator[bool, None, None]:
    n = 0
    while True:
        yield n >= max_iteration
        n = (n + 1) % (max_iteration + 1)

chain = CasualOllamaSingleTurnChain(model="llama3.2:latest", system_prompt=system_prompt, user_prompt_template=user_prompt, base_url="192.168.55.1:11434", request_timeout=300, context_window=32768)
agent = Agent(chain, state_change_callback=state_callback)
loop_agent = LoopAgent(agent, iterator(max_iteration), state_change_callback=state_callback)

In [5]:
query = "Write a short poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=-1, temperature=1.2)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
agent state: running iteration 0
agent state: running iteration 1
agent state: running iteration 2
agent state: running iteration 3
agent state: running iteration 4
agent state: idle
query="Write a short poem with Shakespere's style about the peace before the storm." origin='LoopAgent_2' response='O, fairest calm, thou dost beset my soul,\nA fleeting respite from tempest\'s roll.\nThe sun doth shine, the birds their songs repeat,\nAs if in harmony with Nature\'s beat.\n\nBut soft, what whisperings of wind doth stray\nUpon my ear? A gentle zephyr\'s sway,\nThat stirs the leaves and rustles through the trees,\nA warning sign of tumult\'s soon-to-be.\n\nYet still I bask in calm\'s sweet, golden light,\nUnmindful of the stormy night.\nThe stars, like diamonds in the velvet sky,\nTwinkle with promise of a turbulent sigh.\n\nBut oh, how brief this peace shall prove to be,\nFor like a summer\'s breeze, it whispers "see".\nThe winds of change do stir, and soon I\'ll know\n

In [6]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=1, temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
agent state: running iteration 0
agent state: running iteration 1
agent state: running iteration 2
agent state: running iteration 3
agent state: running iteration 4
agent state: idle
query="Write a poem with Shakespere's style about the peace before the storm." origin='LoopAgent_2' response="O, fairest calm, that doth beset my sight,\nA fleeting peace, 'twixt tumultuous night\nAnd tempests' wrath, that soon shall take its toll,\nLike Zephyr's sigh, that whispers secrets whole.\n\nThe wind doth whisper low, a gentle hush,\nAs petals lift, and morning's dew doth rush,\nThe world, in stillness, waits the coming storm,\nThat will unleash its fury, like a tempest's form.\n\nThe trees, they stand, like sentinels of old,\nTheir branches stretched, as if to grasp the cold\nAir, heavy with the promise of what's to be,\nA reckoning born, of darkness and of sea.\n\nYet, in this calm, I find a strange reprieve,\nA respite from tumult, that doth my soul retrieve,\nA chance to b

In [7]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=3, temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
agent state: running iteration 0
agent state: running iteration 1
agent state: running iteration 2
agent state: running iteration 3
agent state: running iteration 4
agent state: idle
query="Write a poem with Shakespere's style about the peace before the storm." origin='LoopAgent_2' response="O, fairest calm, thou dost entwine thy sway\nUpon the world, in silken, gentle way,\nAs whispers breathe of summer's warmth and light,\nAnd all the senses bask in peaceful night.\n\nThe winds, they doth but softly whisper low,\nOf distant thunderheads that gather slow,\nTheir darkened hearts, aflame with tempests' fire,\nYet calm and tranquil, without one desire\n\nTo burst forth now, and shatter sweet repose,\nThat hath been wrought 'mong verdant, emerald rows.\nThe trees, they stand, unmoved by tempest's sigh,\nAnd leaves, in quiet hush, their vigil keep nigh.\n\nBut thou, dear calm, art but a fleeting guest,\nA peace that hides the stormy passion's nest.\nThou dost beguile, 

In [9]:
from typing import List


def custom_keep_logic(agent_name: str, message: AgentMessage, response: List[str]) -> List[str]:
    return message.responses


query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=custom_keep_logic, temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: running
agent state: running iteration 0
agent state: running iteration 1
agent state: running iteration 2
agent state: idle
query="Write a poem with Shakespere's style about the peace before the storm." origin='LoopAgent_2' response="O, fairest calm of peaceful night's sweet reign,\nWhen whispers of the tempest yet remain\nUnheard, unseen, but in thy quiet breast,\nA murmur stirs, a warning to the rest.\n\nAs sun doth set beneath the western sky,\nAnd twilight's hush doth gently whisper by,\nThe wind, a gentle lover, doth caress\nThe trees with softest touch, and rustling dress.\n\nBut oh, the signs of discord yet appear,\nIn fleeting shadows, dark and faint as fear.\nA chill that creeps, a sense of disarray,\nAs if the very calm did swiftly stray.\n\nAnd I, a mortal, doomed to know the strife,\nDo see in thee, fair peace, a fragile life.\nFor though thou dost enshroud the world in sleep,\nThy presence whispers secrets that do creep\n\nInto my heart, and there do take the